In [7]:
import glob
import pandas as pd
# Use the glob function to match and retrieve the file paths
file_paths = glob.glob('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/**/*.parquet')

# Now, file_paths contains a list of file paths matching the pattern
print(file_paths)

['/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2018/cm_actuals_2018.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2020/cm_actuals_2020.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2021/cm_actuals_2021.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2019/cm_actuals_2019.parquet']


In [8]:
for i in range(0,4):
    df = pd.read_parquet(file_paths[i])
    print(df)

                     outcome
month_id country_id         
457      1               0.0
         2               0.0
         3               0.0
         4               0.0
         5               0.0
...                      ...
468      242             0.0
         243             0.0
         244             0.0
         245             9.0
         246            39.0

[2292 rows x 1 columns]
                     outcome
month_id country_id         
481      1               0.0
         2               0.0
         3               0.0
         4               0.0
         5               0.0
...                      ...
492      242             0.0
         243             0.0
         244             0.0
         245             0.0
         246             0.0

[2292 rows x 1 columns]
                     outcome
month_id country_id         
493      1               0.0
         2               0.0
         3               0.0
         4               0.0
         5            

In [14]:
from sklearn.model_selection import ShuffleSplit


In [15]:
shuffle_split = ShuffleSplit(n_splits=1000, test_size=len(df), random_state=42)


In [16]:
shuffle_split

ShuffleSplit(n_splits=1000, random_state=42, test_size=2292, train_size=None)

In [17]:
for i, (train_idx, _) in enumerate(shuffle_split.split(df)):
    bootstrapped_sample = df.iloc[train_idx]
    # Rename the columns to indicate the bootstrap iteration
    bootstrapped_sample.columns = [f'Bootstrap_{i+1} - {col}' for col in bootstrapped_sample.columns]
    bootstrapped_df = pd.concat([bootstrapped_df, bootstrapped_sample], axis=1)

ValueError: test_size=2292 should be either positive and smaller than the number of samples 2292 or a float in the (0, 1) range

In [31]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import glob
import pyarrow.parquet as pq  

# Use the glob function to match and retrieve the file paths
file_paths = glob.glob('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/**/*.parquet')

# Now, file_paths contains a list of file paths matching the pattern
print(file_paths)
year = 2018
for x in file_paths:
    df = pd.read_parquet(x)
    # Create the original DataFrame
    original_df = pd.DataFrame(df)

    # Initialize an empty DataFrame to store the bootstrapped draws
    bootstrapped_df = pd.DataFrame()

    # Define the number of bootstraps
    n_bootstraps = 1000

    # Perform bootstrapping for each unique combination of country_id and month_id
    for (country_id, month_id), group in original_df.groupby(['country_id', 'month_id']):
        bootstrapped_samples = []
        for _ in range(n_bootstraps):
            bootstrapped_sample = resample(group['outcome'], replace=True)
            bootstrapped_samples.append(bootstrapped_sample.mean())
        bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'country_id': [country_id] * n_bootstraps,
                                                                'month_id': [month_id] * n_bootstraps,
                                                                'draws': bootstrapped_samples})])

    # Set country_id and month_id as indexes
    bootstrapped_df.set_index(['country_id', 'month_id'], inplace=True)
    bootstrapped_df.to_parquet(f'/Users/noorainkazmi/Documents/bootstrap/cm_bootstrapped_{year}',engine='pyarrow')
    # Display the resulting DataFrame
    print(bootstrapped_df)
    year = year + 1


['/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2018/cm_actuals_2018.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2020/cm_actuals_2020.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2021/cm_actuals_2021.parquet', '/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2019/cm_actuals_2019.parquet']
                     draws
country_id month_id       
1          457         0.0
           457         0.0
           457         0.0
           457         0.0
           457         0.0
...                    ...
246        468        39.0
           468        39.0
           468        39.0
           468        39.0
           468        39.0

[2292000 rows x 1 columns]
                     draws
country_id month_id       
1          481         0.0
           481         0.0
           481         0.0
           481         0.0
           481         0.0

In [29]:
bootstrapped_df

draws
country_id month_id       
1          457         0.0
           457         0.0
           457         0.0
           457         0.0
           457         0.0
...                    ...
246        468        39.0
           468        39.0
           468        39.0
           468        39.0
           468        39.0

[2292000 rows x 1 columns]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import glob
import pyarrow.parquet as pq  
def bootstrap_cm_pgm(foldername):
    # Use the glob function to match and retrieve the file paths
    file_paths = glob.glob(foldername+'actuals/cm/**/*.parquet')

    # Now, file_paths contains a list of file paths matching the pattern
    #print(file_paths)
    year = 2018
    for x in file_paths:
        df = pd.read_parquet(x)
        # Create the original DataFrame
        original_df = pd.DataFrame(df)

        # Initialize an empty DataFrame to store the bootstrapped draws
        bootstrapped_df = pd.DataFrame()

        # Define the number of bootstraps
        n_bootstraps = 1000

        # Perform bootstrapping for each unique combination of country_id and month_id
        for (country_id, month_id), group in original_df.groupby(['country_id', 'month_id']):
            bootstrapped_samples = []
            for _ in range(n_bootstraps):
                bootstrapped_sample = resample(group['outcome'], replace=True)
                bootstrapped_samples.append(bootstrapped_sample.mean())
            bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'country_id': [country_id] * n_bootstraps,
                                                                    'month_id': [month_id] * n_bootstraps,
                                                                    'draws': bootstrapped_samples})])

        # Set country_id and month_id as indexes
        bootstrapped_df.set_index(['country_id', 'month_id'], inplace=True)
        bootstrapped_df.to_parquet(f'{foldername}/bootstrap/cm_bootstrapped_{year}',engine='pyarrow')
        # Display the resulting DataFrame
        year = year+1
        
    file_paths = glob.glob(foldername+'actuals/pgm/**/*.parquet')

    # Now, file_paths contains a list of file paths matching the pattern
    #print(file_paths)
    year = 2018
    for x in file_paths:
        df = pd.read_parquet(x)
        # Create the original DataFrame
        original_df = pd.DataFrame(df)

        # Initialize an empty DataFrame to store the bootstrapped draws
        bootstrapped_df = pd.DataFrame()

        # Define the number of bootstraps
        n_bootstraps = 1000

        # Perform bootstrapping for each unique combination of country_id and month_id
        for (country_id, month_id), group in original_df.groupby(['priogrid_gid', 'month_id']):
            bootstrapped_samples = []
            for _ in range(n_bootstraps):
                bootstrapped_sample = resample(group['outcome'], replace=True)
                bootstrapped_samples.append(bootstrapped_sample.mean())
            bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'priogrid_gid': [priogrid_gid] * n_bootstraps,
                                                                    'month_id': [month_id] * n_bootstraps,
                                                                    'draws': bootstrapped_samples})])

        # Set country_id and month_id as indexes
        bootstrapped_df.set_index(['priogrid_gid', 'month_id'], inplace=True)
        bootstrapped_df.to_parquet(f'{foldername}/bootstrap/pgm_bootstrapped_{year}',engine='pyarrow')
        year = year + 1
        # Display the resulting DataFrame

In [38]:
df = pd.read_parquet('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/features/cm_features_to_oct2017.parquet')

In [39]:
df

gleditsch_ward  ged_sb  ged_ns  ged_os  acled_sb  \
month_id country_id                                                     
121      1                      110     0.0     0.0     0.0       0.0   
         2                      115     0.0     0.0     0.0       0.0   
         3                       52     0.0     0.0     0.0       0.0   
         4                      101     0.0     0.0    12.0       0.0   
         5                      990     0.0     0.0     0.0       0.0   
...                             ...     ...     ...     ...       ...   
454      242                    510     0.0     0.0     0.0       1.0   
         243                    600     0.0     0.0     0.0       0.0   
         244                    435     0.0     0.0     0.0       0.0   
         245                    625     0.0    36.0     1.0       7.0   
         246                    626    19.0     3.0     0.0     207.0   

                     acled_sb_count  acled_os  ged_sb_tsum_24  \
month_id country_id                                             
121      1                      0.0       0.0             0.0   
         2                      0.0       0.0             0.0   
         3                      0.0       0.0             0.0   
         4                      0.0       0.0             0.0   
         5                      0.0       0.0             0.0   
...                             ...       ...             ...   
454      242                    1.0       0.0             0.0   
         243                    0.0       0.0             0.0   
         244                    0.0       0.0             0.0   
         245                    5.0      39.0          1529.0   
         246                   38.0      66.0          1096.0   

                     wdi_sp_pop_totl  ged_sb_tlag_1  ...  ind_efficiency_t48  \
month_id country_id                                  ...                       
121      1                  752897.0            0.0  ...           43.141952   
         2                  408360.0            0.0  ...           20.844640   
         3                 1256210.0            0.0  ...           42.245824   
         4                19275292.0            0.0  ...          159.866899   
         5                  167886.0            0.0  ...            0.000000   
...                              ...            ...  ...                 ...   
454      242              54401802.0            0.0  ...          412.971789   
         243              35107264.0            0.0  ...          106.996363   
         244               4051890.0            0.0  ...           56.575372   
         245              39377169.0           19.0  ...          161.387656   
         246              11066105.0           32.0  ...           12.105882   

                     irr_agr_efficiency_t48  services_efficiency_t48  \
month_id country_id                                                    
121      1                         0.151779                27.952694   
         2                         0.722621                57.289771   
         3                         1.671158                35.755839   
         4                         0.443153                28.946372   
         5                         0.000000                 0.000000   
...                                     ...                      ...   
454      242                       0.132565                33.875609   
         243                       0.380566                46.472747   
         244                       0.200425                23.185089   
         245                       0.203982                38.842079   
         246                       0.018451                10.037360   

                     general_efficiency_t48  water_stress_t48  \
month_id country_id                                             
121      1                         1.380676          3.333333   
         2                         3.922495         

In [37]:
import pandas as pd
import numpy as np

# Sample data for demonstration
data = {
    'month_id': [1, 2, 1, 2, 1, 2],
    'country_id': ['A', 'A', 'B', 'B', 'C', 'C'],
    'outcome': [10, 15, 20, 25, 30, 35]
}

# Create the original DataFrame
original_df = pd.read_parquet(file_paths[0])

# Initialize an empty DataFrame to store the bootstrapped draws
bootstrapped_df = pd.DataFrame()

# Define the number of bootstraps
n_bootstraps = 1000

# Perform bootstrapping for each unique combination of month_id and country_id
for (month_id, country_id), group in original_df.groupby(['month_id', 'country_id']):
    for draw in range(n_bootstraps):
        bootstrapped_sample = group['outcome'].sample(frac=1, replace=True)
        bootstrapped_df = pd.concat([bootstrapped_df, pd.DataFrame({'month_id': [month_id],
                                                                   'country_id': [country_id],
                                                                   'draw': [draw],
                                                                   'outcome': [bootstrapped_sample.mean()]})])

# Set month_id, country_id, and draw as indexes
bootstrapped_df.set_index(['month_id', 'country_id', 'draw'], inplace=True)

# Display the resulting DataFrame
print(bootstrapped_df)


KeyboardInterrupt: 

In [40]:
import pandas as pd
import argparse

def copy_parquet_file(input_file, output_file):
    try:
        # Read the input Parquet file
        df = pd.read_parquet(input_file)

        # Add transformation code here
        
        
        # Save the DataFrame to the output Parquet file
        df.to_parquet(output_file, index=False)

        print(f"Parquet file '{input_file}' copied to '{output_file}' successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Copy a Parquet file.")
    parser.add_argument("input_file", help="Path to the input Parquet file.")
    parser.add_argument("output_file", help="Path to the output Parquet file.")

    args = parser.parse_args()

    copy_parquet_file(args.input_file, args.output_file)


usage: ipykernel_launcher.py [-h] input_file output_file
ipykernel_launcher.py: error: the following arguments are required: input_file, output_file


SystemExit: 2

/Users/noorainkazmi/miniforge3/envs/prediction_c/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [66]:
df = pd.read_parquet('/Users/noorainkazmi/Dropbox (ViEWS)/competition_data/actuals/cm/test_window_2018/cm_actuals_2018.parquet')

In [67]:
from functools import partial

def bootstrap_expand_single_point_predictions(ensemble_df, draw_from_column, level, year_list, draws=1000):
    """
    Expands an input prediction df with one prediction per unit to n draws from the 'draw_from' array,
    bootstrap-fashion, and returns a list of dictionaries with prediction and metadata for all years in year_list
    """

    actuals = ensemble_df[draw_from_column].fillna(0)
    actuals_by_year = []
    for year in year_list:
        actuals_dict = {
            'year': year,
            'actuals_df': extract_year(year=year, df=actuals)
        }
        actuals_by_year.append(actuals_dict)

    # Expanding by drawing n draws from specified draw_from array

    for year_record in actuals_by_year:
        print(year_record['year'])
        df = year_record.get('actuals_df')
        year_record['expanded_df'] = expanded_df_bootstrap(df, ndraws=draws, draw_from=df[draw_from_column],
                                                           level=level)
    return actuals_by_year

def expanded_df_bootstrap(df, ndraws=1000, draw_from=None, level='cm'):
    function_with_draws = partial(sample_bootstrap_row, draw_from=draw_from, ndraws=ndraws)
    df['draws'] = df.apply(function_with_draws, axis=1)
    exploded_df = df.explode('draws').astype('int32')
    if level == 'cm':
        exploded_df['draw'] = exploded_df.groupby(['month_id', 'country_id']).cumcount()
    if level == 'pgm':
        exploded_df['draw'] = exploded_df.groupby(['month_id', 'priogrid_gid']).cumcount()
    exploded_df.drop(columns=['outcome'], inplace=True)
    exploded_df.rename(columns={'draws':'outcome'}, inplace=True)
    exploded_df.set_index('draw', append=True, inplace=True)
    return exploded_df

def extract_year(year, df):
    """
    Extract data from input dataframe in year-chunks
    """
    first_month = (year - 1980)*12 + 1
    last_month = first_month + 11
    return pd.DataFrame(df.loc[first_month:last_month])

def sample_bootstrap_row(row: pd.DataFrame, draw_from: np.array, ndraws: int = 100) -> np.ndarray:
    """Given a dataframe row, produce ndraws draws from the prediction column in the df.
    Attention, this is a row vectorized approach, should be used with apply.
    :return an np array. Should be exploded for long df.
    """
    return np.random.choice(draw_from, size=ndraws, replace=True)

def save_models(level,model_names,model_list, filepath):
    ''' Saves the models to dropbox '''
    
    i = 0
    for bm_model in model_list:
        for record in bm_model:
            year_record = record # First part of record list is list of yearly predictions, second is string name for benchmark model
            print(year_record['year'])
            filename = filepath + 'bm_' + level + '_' + model_names[i] + '_expanded_' + str(year_record['year']+1) + '.parquet'
            print(filename)
            year_record['expanded_df'].to_parquet(filename)
        i = i + 1

In [68]:
year_list = [2018]
actual_pgm = bootstrap_expand_single_point_predictions(df,draw_from_column='outcome',level='cm',year_list=year_list,draws=1000)

2018


In [69]:
actual_pgm

[{'year': 2018,
  'actuals_df':                      outcome  \
  month_id country_id            
  457      1               0.0   
           2               0.0   
           3               0.0   
           4               0.0   
           5               0.0   
  ...                      ...   
  468      242             0.0   
           243             0.0   
           244             0.0   
           245             9.0   
           246            39.0   
  
                                                                   draws  
  month_id country_id                                                     
  457      1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
           2           [2301.0000000000005, 0.0, 0.0, 0.0, 0.0, 14.99...  
           3           [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 42.0,...  
           4           [148.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.00000000000...  
           5           [89.99999999999997, 0.0, 0.0, 0.0, 0.0, 0.0, 0...  


In [70]:
model_names = ['bootstrap']
model_list = [actual_pgm]
filepath = '/Users/noorainkazmi/Documents/boots@/'
save_models('cm',model_names,model_list, filepath)

2018
/Users/noorainkazmi/Documents/boots@/bm_pgm_bootstrap_expanded_2019.parquet


In [62]:
df = pd.read_parquet('/Users/noorainkazmi/Documents/boots@bm_pgm_bootstrap_expanded_2018.parquet')

In [63]:
df

outcome
month_id country_id draw         
457      1          0           0
                    1           0
                    2           0
                    3           0
                    4           0
...                           ...
468      246        995         0
                    996         0
                    997         0
                    998         0
                    999         0

[2292000 rows x 1 columns]

In [72]:
df['outcome']>0

month_id  country_id
457       1             False
          2             False
          3             False
          4             False
          5             False
                        ...  
468       242           False
          243           False
          244           False
          245            True
          246            True
Name: outcome, Length: 2292, dtype: bool

In [73]:
import pandas as pd
import argparse
from functools import partial

def bootstrap_expand_single_point_predictions(ensemble_df, draw_from_column, level, year_list, draws=1000):
    """
    Expands an input prediction df with one prediction per unit to n draws from the 'draw_from' array,
    bootstrap-fashion, and returns a list of dictionaries with prediction and metadata for all years in year_list
    """

    actuals = ensemble_df[draw_from_column].fillna(0)
    actuals_by_year = []
    for year in year_list:
        actuals_dict = {
            'year': year,
            'actuals_df': extract_year(year=year, df=actuals)
        }
        actuals_by_year.append(actuals_dict)

    # Expanding by drawing n draws from specified draw_from array

    for year_record in actuals_by_year:
        print(year_record['year'])
        df = year_record.get('actuals_df')
        year_record['expanded_df'] = expanded_df_bootstrap(df, ndraws=draws, draw_from=df[draw_from_column],
                                                           level=level)
    return actuals_by_year

def expanded_df_bootstrap(df, ndraws=1000, draw_from=None, level='cm'):
    function_with_draws = partial(sample_bootstrap_row, draw_from=draw_from, ndraws=ndraws)
    df['draws'] = df.apply(function_with_draws, axis=1)
    exploded_df = df.explode('draws').astype('int32')
    if level == 'cm':
        exploded_df['draw'] = exploded_df.groupby(['month_id', 'country_id']).cumcount()
    if level == 'pgm':
        exploded_df['draw'] = exploded_df.groupby(['month_id', 'priogrid_gid']).cumcount()
    exploded_df.drop(columns=['outcome'], inplace=True)
    exploded_df.rename(columns={'draws':'outcome'}, inplace=True)
    exploded_df.set_index('draw', append=True, inplace=True)
    return exploded_df

def extract_year(year, df):
    """
    Extract data from input dataframe in year-chunks
    """
    first_month = (year - 1980)*12 + 1
    last_month = first_month + 11
    return pd.DataFrame(df.loc[first_month:last_month])

def sample_bootstrap_row(row: pd.DataFrame, draw_from: np.array, ndraws: int = 100) -> np.ndarray:
    """Given a dataframe row, produce ndraws draws from the prediction column in the df.
    Attention, this is a row vectorized approach, should be used with apply.
    :return an np array. Should be exploded for long df.
    """
    return np.random.choice(draw_from, size=ndraws, replace=True)

def save_models(level,model_names,model_list, filepath):
    ''' Saves the models to dropbox '''
    
    i = 0
    for bm_model in model_list:
        for record in bm_model:
            year_record = record # First part of record list is list of yearly predictions, second is string name for benchmark model
            print(year_record['year'])
            filename = filepath + 'bm_' + level + '_' + model_names[i] + '_expanded_' + str(year_record['year']+1) + '.parquet'
            print(filename)
            year_record['expanded_df'].to_parquet(filename)
        i = i + 1
        
def copy_parquet_file(input_file, output_file,year,cmorpgm):
    try:
        # Read the input Parquet file
        df = pd.read_parquet(input_file)

        # Add transformation code here
        year_list = [year]
        actual_pgm = bootstrap_expand_single_point_predictions(df,draw_from_column='outcome',level=cmorpgm,year_list=year_list,draws=1000) 
        model_names = ['bootstrap']
        model_list = [actual_pgm]
        filepath = '/Users/noorainkazmi/Documents/boots@/'
        save_models(cmorpgm,model_names,model_list, filepath + output_file)
        
        # Save the DataFrame to the output Parquet file
        df.to_parquet(output_file, index=False)
        print(f"Parquet file '{input_file}' copied to '{output_file}' successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Copy a Parquet file.")
    parser.add_argument("input_file", help="Path to the input Parquet file.")
    parser.add_argument("output_file", help="Path to the output Parquet file.")
    parser.add_argument('year_of_actuals',help="Year of actuals")
    parser.add_argument('cm_or_pgm',help='Enter cm or pgm based on file name')
    args = parser.parse_args()
    copy_parquet_file(args.input_file, args.output_file,args.year_of_actuals,args.cm_or_pgm)


usage: ipykernel_launcher.py [-h]
                             input_file output_file year_of_actuals cm_or_pgm
ipykernel_launcher.py: error: the following arguments are required: input_file, output_file, year_of_actuals, cm_or_pgm


SystemExit: 2

/Users/noorainkazmi/miniforge3/envs/prediction_c/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
